In [1]:
##Estimación de indicadores de población ocupada en cultura
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr)

Loading required package: pacman



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
url<-"https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2024_trim2_csv.zip"


In [4]:
# Descargar el archivo y descomprimirlo
options(timeout = 9999999)
destfile <- "data/conjunto_de_datos_enoe_2024_2t_csv.zip"
download.file(url, destfile = destfile)
unzip(destfile, exdir = "data")

In [5]:
#Leer tabla y filtrar ocupados

ocupados<-read_csv("data/ENOE_SDEMT224.csv", show_col_types = FALSE)%>%
janitor::clean_names()%>%
filter(
       clase2==1,
       ((eda>=15 & eda<=98 & !is.na(eda)) &
                                     (c_res==1 | c_res==3)))
nrow(ocupados)


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


[1] 195278

In [6]:
#Traer clasificación SINCO

sinco<-read_csv("data/ENOE_COE1T224.csv", show_col_types = FALSE)%>%janitor::clean_names()%>%
select(tipo, mes_cal,cd_a,ent,con,v_sel,n_hog,h_mud, n_ren,p3)%>%
#Renombrar p3 a sinco
rename(sinco = p3)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [7]:
#Unir tablas
ocupados<-left_join(ocupados, sinco, by = c("tipo", "mes_cal", "cd_a", "ent", "con", "v_sel", "n_hog", "h_mud", "n_ren"))%>%
#Clasificar empleo cultural usando la clasificación SINCO y metodología de cultura federal
mutate(cultura_federal=case_when(sinco %in% c(2175,7611,2552,2131,7313,7342,7321,2174,7343,2173,2171,1721,1722,
2544,2271,2625,2162,1422,1322,1421,2541,2543,2542,2164,2151,2163,2144,2425,2655,3222,7322,2712,
7223,2551,5212,2636,4233,2172,8155,8144,7999,7344,2553,7352,2152,2161,7323,7201,7601,7401,7301,
2652,7332,9713,4312,7614,7353) ~ 1,
         TRUE ~ 0))%>%
#Clasificar empleo cultural de acuerdo con metodología de Cultura de la Ciudad de México
mutate(cultura_cdmx=case_when(sinco %in% c(2175,7611,2552,2131,7313,7342,7321,2174,7343,2173,2171,1721,1722,
2544,2271,2625,2162,1422,1322,1421,2541,2543,2542,2164,2151,2163,2144,2425,2655,3222,7322,2712,
7223,2551,5212,2636,4233,2172,8155,8144,7999,7344,2553,7352,2152,2161,7323,7201,7601,7401,7301,
2652,7332,9713,4312,7614,7353,2514,5212,2653,2654,7299,7999,7311,7331,7341,7411,7412,9234,9235,
                                           9237,4233,2141,2153,2339,2334,2715,1412,1712,3221,5101,
5111,5112,5113,5114,5115,5116,5119,7515,7517,9411

) ~ 1,
         TRUE ~ 0))

nrow(ocupados)

[1] 195278

In [8]:
#Opción para ajustar si se tiene una sola upm
options(survey.lonely.psu="certainty")
dm<-ocupados%>%
as_survey_design(ids = upm,
                 strata = est_d_tri,
                 weights = fac_tri,
                    nest = TRUE)

In [9]:
#Total de ocupados en cdmx
dm%>%
summarise(ocupados = survey_total(vartype = "cv"))

ocupados,ocupados_cv
<dbl>,<dbl>
59327161,0.004876444


In [11]:
#Estimación de ocupados en cultura
print("Población ocupada en cultura por entidad federativa de acuerdo a metodología de cultura federal")
dm%>%
filter(cultura_federal==1)%>%
  group_by(ent)%>%
summarise(ocupados = survey_total(vartype = "cv"),
porcentaje = survey_prop(vartype = "cv"))%>%
mutate(porcentaje = porcentaje*100)%>%
janitor::adorn_totals("row")

[1] "Población ocupada en cultura por entidad federativa de acuerdo a metodología de cultura federal"


ent,ocupados,ocupados_cv,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,22263,0.08518543,1.0015147,0.08852872
2,56137,0.09053163,2.5253573,0.09223963
3,13960,0.10644202,0.6279991,0.10915002
4,12479,0.09821561,0.5613754,0.10131205
5,36128,0.08418978,1.6252402,0.08708418
6,10885,0.13776562,0.4896684,0.13971264
7,45061,0.22704909,2.0270966,0.22402551
8,54314,0.08279405,2.4433485,0.08512348
9,266496,0.07635535,11.9884855,0.07182184


In [14]:
#Población ocupada en cultura en la Ciudad de México

print("Población ocupada en cultura en la Ciudad de México de acuerdo a metodología de cultura cdmx")
dm%>%
  filter(ent==9)%>%
  group_by(cultura_federal)%>%
    summarise(ocupados = survey_total(vartype = "cv"),
                porcentaje = survey_prop(vartype = "cv"))%>%
  mutate(porcentaje = porcentaje*100)%>%
  filter(cultura_federal==1)

[1] "Población ocupada en cultura en la Ciudad de México de acuerdo a metodología de cultura cdmx"


cultura_federal,ocupados,ocupados_cv,porcentaje,porcentaje_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,266496,0.07635535,5.512677,0.07551563


In [13]:
#Población ocupada en cultura en la Ciudad de México

print("Población ocupada en cultura en la Ciudad de México de acuerdo a metodología de cultura cdmx")
dm%>%
  filter(ent==9)%>%
  group_by(cultura_cdmx)%>%
    summarise(ocupados = survey_total(vartype = "cv"),
                porcentaje = survey_prop(vartype = "cv"))%>%
  mutate(porcentaje = porcentaje*100)%>%
  filter(cultura_cdmx==1)


[1] "Población ocupada en cultura en la Ciudad de México de acuerdo a metodología de cultura cdmx"


cultura_cdmx,ocupados,ocupados_cv,porcentaje,porcentaje_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,718802,0.04681013,14.86898,0.0434248
